In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "openchat/openchat-3.5-1210"  # Hoặc bản openchat/openchat-3.5-0106

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
from datasets import load_dataset

# Tải dataset
dataset = load_dataset("hungnm/vietnamese-medical-qa")

# Kiểm tra cấu trúc dataset
print(dataset)

# Lấy cột question và answer, bổ sung prompt reasoning
def add_reasoning_prompt(example):
    # Prompt ép COT reasoning
    template = (
        "Bạn là bác sĩ AI. Hãy giải thích từng bước reasoning khi trả lời câu hỏi dưới đây.\n"
        "Hỏi: {question}\n"
        "Trả lời:"
    )
    example["prompt"] = template.format(question=example["question"])
    example["output"] = example["answer"]
    return example

dataset = dataset.map(add_reasoning_prompt)


DatasetDict({
    train: Dataset({
        features: ['answer', 'question'],
        num_rows: 9335
    })
})


In [ ]:
from transformers import DataCollatorForLanguageModeling

def preprocess_sft(example):
    prompt = example["prompt"]
    output = example["output"]
    # Dạng prompt: <prompt> <output>
    text = prompt + " " + output
    tokens = tokenizer(text, truncation=True, max_length=1024)
    return tokens

sft_dataset = dataset["train"].map(preprocess_sft, remove_columns=dataset["train"].column_names)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


Map:   0%|          | 0/9335 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Cấu hình LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                # Rank, để 8 hoặc 16 tùy VRAM
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]  # Thường là như này với Llama/OpenChat (có thể cần đổi)
)

# Gắn LoRA adapter lên model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Xem có bao nhiêu tham số trainable


trainable params: 3,407,872 || all params: 7,245,156,352 || trainable%: 0.0470


In [ ]:
from transformers import TrainingArguments, Trainer
import os
os.environ['WANDB_DISABLED'] = 'true'

training_args = TrainingArguments(
    output_dir="sft-openchat-medqa",
    per_device_train_batch_size=4,      # test batch 2/4
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=1,
    bf16=True,
    save_steps=500,
    logging_steps=100,
    save_total_limit=1,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sft_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


<ipython-input-13-c459ce2713ea>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,1.355900
200,1.218800
300,1.154300
400,1.134700
500,1.124200
600,1.113600
700,1.100500
800,1.106300
900,1.116100
1000,1.115800


TrainOutput(global_step=1167, training_loss=1.146103879377795, metrics={'train_runtime': 1676.836, 'train_samples_per_second': 5.567, 'train_steps_per_second': 0.696, 'total_flos': 3.2890546726539264e+17, 'train_loss': 1.146103879377795, 'epoch': 1.0})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# question = "Thưa bác sĩ, em gần đây hay bị mỏi mắt thì nên lưu ý những gì?"
prompt = """
      Bạn là bác sĩ AI. Hãy giải thích từng bước reasoning khi trả lời câu hỏi dưới đây.

      Ví dụ:
      Hỏi: Bệnh nhân có sốt, ho, đau họng. Bạn nghĩ bệnh nhân mắc bệnh gì?
      Trả lời: Đầu tiên, các triệu chứng sốt, ho và đau họng thường gặp ở các bệnh lý đường hô hấp như cảm cúm hoặc viêm họng. Tiếp theo, nếu bệnh nhân có yếu tố dịch tễ như tiếp xúc người bệnh hoặc thay đổi thời tiết, khả năng cảm cúm càng cao. Vì vậy, khả năng cao bệnh nhân mắc cảm cúm.

      ---
      Hỏi: Gần đây em hay bị sốt nhẹ, ho, đau họng. Bác sĩ nghĩ em mắc bệnh gì?
      Trả lời:
      """

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,         # <-- Quan trọng!
    top_p=0.9,
    temperature=0.6
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



      Bạn là bác sĩ AI. Hãy giải thích từng bước reasoning khi trả lời câu hỏi dưới đây.

      Ví dụ:
      Hỏi: Bệnh nhân có sốt, ho, đau họng. Bạn nghĩ bệnh nhân mắc bệnh gì?
      Trả lời: Đầu tiên, các triệu chứng sốt, ho và đau họng thường gặp ở các bệnh lý đường hô hấp như cảm cúm hoặc viêm họng. Tiếp theo, nếu bệnh nhân có yếu tố dịch tễ như tiếp xúc người bệnh hoặc thay đổi thời tiết, khả năng cảm cúm càng cao. Vì vậy, khả năng cao bệnh nhân mắc cảm cúm.

      ---
      Hỏi: Gần đây em hay bị sốt nhẹ, ho, đau họng. Bác sĩ nghĩ em mắc bệnh gì?
      Trả lời:
      1. Bạn nên đi khám để xác định bệnh nguy hiểm.
      2. Bạn nên ăn uống nhiều rau củ quả, nhiều nước, uống thuốc chống viêm họng, thuốc tăng cường huyết lượng, thuốc tăng cường kháng sinh.
      3. Bạn nên dùng thuốc điều trị sốt và ho như Paracetamol, Ibuprofen, Cetirizine...
      4. Bạn nên dùng thuốc điều trị đau họng như Paracetamol, Ibuprofen, Cetirizine...
      Bạn có thể đi khám để được bác sĩ tư vấn thêm
